In [1]:
using TSSOS
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials
using QuadGK
using JuMP
using Random
using NLopt

## Define the quantum system

In [2]:
# We look at a typical Hamiltonian

#Drift Hamiltonian

H0 = [
    0 0 0;
    0 0.515916 0;
    0 0 1
];

H0 ./= norm(H0, Inf)

# Control Hamiltonian
V = [
    0 0.707107 0;
    0.707107 0 1;
    0 1 0
]

V ./= norm(V, Inf);

In [3]:
H0*V-V*H0

3×3 Matrix{Float64}:
 0.0       -0.364808   0.0
 0.364808   0.0       -0.484084
 0.0        0.484084   0.0

## Useful functions with polynomials

In [4]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

function square_frobenius_norm2(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    sum(z' * z for z in M)
end

square_frobenius_norm2 (generic function with 1 method)

## Magnus expansion

In [5]:
@polyvar x[1:3]
@polyvar t[1:3]

# final time
const T = 0.5

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end

function A(t, x,)
    #=
    The generator of motion entering the Magnus expansion
    =#
    (H0 + V * u(t, x)) / im
end

function commutator(a, b)
    a * b - b * a
end 

# get the partial sum of the Magnus expansion
A₁ = A(t[1], x)
A₂ = A(t[2], x)

Ω = ∫(A₁, t[1], 0, T);

# 2nd term in the Magnus expansion
Ω .+= 1//2 * ∫(∫(
    commutator(A₁, A₂), 
    t[2], 0, t[1]), 
    t[1], 0, T
);

# 3nd term in the Magnus expansion

A₃ = A(t[3], x)

Ω .+= 1//6 * ∫(∫(∫(
    commutator(A₁, commutator(A₂, A₃)) + commutator(commutator(A₁, A₂), A₃),
    t[3], 0, t[2]),
    t[2], 0, t[1]),
    t[1], 0, T
);


## Get random unitary target and take logarithm

In [6]:
function get_unitary(x::AbstractArray)
    #=
    Get the unitary given the coefficients for the polynomial control
    =#
    basis = NLevelBasis(size(H0)[1])

    𝓗₀ = DenseOperator(basis, basis, H0)
    𝓥 = DenseOperator(basis, basis, V)

    H = LazySum([1., u(0, x)], [𝓗₀, 𝓥])
        
    function 𝓗(t, psi)
        H.factors[2] = u(t, x)
        return H
    end

    _, 𝓤 = timeevolution.schroedinger_dynamic([0, T], identityoperator(basis, basis), 𝓗)
    
    return Matrix(𝓤[2].data)
end

get_unitary (generic function with 1 method)

In [7]:
function logarithm_mat(U)
    F= eigen(U)
    X = F.vectors
    L = Diagonal(F.values)
    n= 0
    theta_values = [log(λ) for λ in F.values]
    K = Diagonal(theta_values + 2*pi*im*n*[1,1,1])
    X*K*inv(X)
end

logarithm_mat (generic function with 1 method)

## Functions to compare

In [8]:
function Magnus(x)
    # get the partial sum of the Magnus expansion
    A₁ = A(t[1], x)
    A₂ = A(t[2], x)
    
    Ω = ∫(A₁, t[1], 0, T);
    
    # 2nd term in the Magnus expansion
    Ω .+= 1//2 * ∫(∫(
        commutator(A₁, A₂), 
        t[2], 0, t[1]), 
        t[1], 0, T
    );
    
    # 3nd term in the Magnus expansion
    
    A₃ = A(t[3], x)
    
    Ω .+= 1//6 * ∫(∫(∫(
        commutator(A₁, commutator(A₂, A₃)) + commutator(commutator(A₁, A₂), A₃),
        t[3], 0, t[2]),
        t[2], 0, t[1]),
        t[1], 0, T
    );
    Ω

end

Magnus (generic function with 1 method)

In [9]:
function local_minimize(obj::AbstractPolynomial, init_x::AbstractArray)
    #=
    Perform local minimization of obj polynomial using init_x as initial guess
    =#
    vars = variables(obj)

    @assert length(vars) == length(init_x)
    
    function g(a...)
        # Converting polynomial expression to function to be minimize
        obj(vars => a)
    end
    
    model = Model(NLopt.Optimizer)

    set_optimizer_attribute(model, "algorithm", :LD_MMA)

    set_silent(model)
    @variable(model, y[1:length(vars)])

    # set initial guess
    for (var, init_val) in zip(y, init_x)
        set_start_value(var, init_val)
    end

    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))
    JuMP.optimize!(model)

    map(value, y)
end

local_minimize (generic function with 1 method)

## TSSOS

In [36]:
@time begin
    
n_samples = 1000
Random.seed!(6292022)

# randomly generate the coefficients for the polynomial control 
exact_x = -1 .+ 2 * rand(length(x) * n_samples)
exact_x = reshape(exact_x, (length(x), n_samples))

# The values of objective functions for exact x
obj_exact_x = zeros(n_samples)
    
# The array of target unitaries synthesized by the control x
U_targets = zeros(ComplexF64, n_samples, size(H0)...)

# the polynomial objective at min_x
glob_obj_min_x = zeros(n_samples)

# The global minimum via TSSOS library
tssos_glob_obj_min = zeros(n_samples)

# Frobenius norm difference between target and obtained unitaries
norm_U_target_minus_obtained = zeros(n_samples)

# Frobenius norm difference between target and the truncated Magnus expansion
norm_U_target_minus_expΩ_exact_x = zeros(n_samples)
norm_U_target_minus_expΩ_min_x = zeros(n_samples)

# The normalised overlap of the evolution and the target 
f_PSU = zeros(n_samples) 

# Convergence test for the Magnus expansion for the exact control (convergence_test_exact_x < 1)
convergence_test_exact_x = zeros(n_samples)


# Convergence test for the Magnus expansion for the obtained control (convergence_test_min_x < 1)
convergence_test_min_x = zeros(n_samples)

norm_diff_x = zeros(n_samples)

Threads.@threads for i=1:n_samples
    
    # target unitray
    U_targets[i, :, :] = U_target = get_unitary(exact_x[:, i])
        
    # get the polynomial objective function
    obj = square_frobenius_norm(
        Ω - logarithm_mat(U_targets[i,:,:])
    )
    
    # save the value of objective function for exact x
    obj_exact_x[i] = obj(exact_x[:, i])
  
    
    # Get the global minimum via TSSOS library
    opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true)
    
    previous_sol = sol
    previous_opt = opt
    
    while ~isnothing(sol)
        previous_sol = sol
        previous_opt = opt
            
        opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
    end
    tssos_glob_obj_min[i] = previous_opt
    min_x = previous_sol
    
    # refine the estimate by local minimization
    #min_x = local_minimize(obj, min_x)
    
    # the polynomial objective at min_x
    glob_obj_min_x[i] = tssos_glob_obj_min[i]
        
    # get the Frobenius norm difference between target and obtained unitaries
    U_star = get_unitary(min_x)
    norm_U_target_minus_obtained[i] = norm(U_target - U_star)

    # The normalised overlap of the evolution and the target 
    f_PSU[i] = abs(tr(U_target' * U_star)) / size(U_star)[1]
        
    # check the accuracy of the Magnus expansion
    Ω_exact_x = convert(Matrix{ComplexF64}, subs(Ω, x=>exact_x[:, i]))
    norm_U_target_minus_expΩ_exact_x[i] = norm(U_target - exp(Ω_exact_x))

    Ω_min_x = convert(Matrix{ComplexF64}, subs(Ω, x=>min_x))
    norm_U_target_minus_expΩ_min_x[i] = norm(U_target - exp(Ω_min_x))
    
    # Convergence test for the Magnus expansion for the exact control
    convergence_test_exact_x[i] = quadgk(t -> opnorm(A(t, exact_x[:, i])), 0, T)[1] / π

    # Convergence test for the Magnus expansion for the obtained control
    convergence_test_min_x[i] = quadgk(t -> opnorm(A(t, min_x)), 0, T)[1] / π    
    
    norm_diff_x[i] = norm(exact_x[:, i] - min_x)
end

end

*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS ***********************************
*********************************** TSSOS **********************

In [37]:
using HDF5


h5open("results.hdf5", "w") do fid
    fid["U_targets"] = U_targets
    fid["exact_x"] = exact_x
    fid["obj_exact_x"] = obj_exact_x
    fid["tssos_glob_obj_min"] = tssos_glob_obj_min
    fid["norm_U_target_minus_obtained"] = norm_U_target_minus_obtained
    fid["f_PSU"] = f_PSU
    fid["convergence_test_exact_x"] = convergence_test_exact_x
    fid["convergence_test_min_x"] = convergence_test_min_x
end;